# Stitching back together - function

In [6]:
# We have an input of a numpy array with one dimension being: 'bounding box'
# shape (num_rows, 4)

In [15]:
import numpy as np
import pandas as pd

In [16]:
ex_bbs = np.random.randint(70, 640, (10, 4))
ex_bbs

array([[442, 434, 277, 102],
       [354, 592, 344, 350],
       [613, 119, 131, 467],
       [267, 109, 271, 363],
       [560, 103, 174,  70],
       [510, 568, 595, 611],
       [504, 466, 530, 395],
       [194, 240, 477, 629],
       [621,  79, 237, 468],
       [225, 154, 406, 314]])

# Function: **luc_coordinates**

In [17]:
def luc_coordinates(num_width_comp, num_height_comp, comp_size):
  ''' A function that takes as an input:
  width_comp = number of compartments for the width (e.g. 8)
  height_comp = number of compartments for the height (e.g. 6)
  comp_size = size of each compartment (e.g. 640)
  and returns the coordinates of the left-upper-corner (luc) of
  each compartment
  '''

  # Height and width of the input image
  width = 4000
  height = 3000

  # Step size is the size of the compartments unique pixels horizontally and vertically
  step_size_horiz = width / num_width_comp
  step_size_vert = height / num_height_comp

  # Defining how much overlap there will be between each compartment vertically and horizontally
  overlap_horiz = int(((comp_size * num_width_comp) - width)/num_width_comp)
  overlap_vert = int(((comp_size * num_height_comp) - height)/num_height_comp)

  #creating a list with all the x_coordinates for the compartments (8)
  x_coordinates = [0]
  [x_coordinates.append(int(ele * step_size_horiz)) for ele in range(1, num_width_comp)]

  # creating a list with all the y_coordinates for the compartments (6)
  y_coordinates = [0]
  [y_coordinates.append(int(ele * step_size_vert)) for ele in range(1, num_height_comp)]


  # Creating a dictionary with the coordinates
  coordinates_dict = {}
  b = 0 # no inherent important meaning

  for ele in range (0, num_height_comp):
    for i in range (0, num_width_comp):
      a = [x_coordinates[i], y_coordinates[ele]]
      b += 1
      coordinates_dict[f'cor_{b}'] = a

  return coordinates_dict

# Function: **slicing_dictionary**

In [18]:
def slicing_dictionary(tensor):

  # Calling the function 'luc_coordinates' and saving the resulting dictionary of coordinates in a variable
  # coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

  slize_size = 640

  # Turning the coordinates into a list
  coordinates_list = [value for key, value in coordinates_dict.items()]

  # creating a for loop for getting the correct slizing integers and putting them into a dictionary
  slicing_dict = {}

  for i in range (0, 48):
    slice_1, slice_2 = coordinates_list[i][0], coordinates_list[i][1]
    slice_1_a = slice_1
    slice_1_b = slice_1_a + slize_size
    slice_2_a = slice_2
    slice_2_b = slice_2_a + slize_size
    slicing_dict[i] = [slice_1_a, slice_1_b, slice_2_a, slice_2_b]

  return slicing_dict

# Function: pipeline_input_to_slicing_dict

In [19]:
# Pipeline from Input-tensor to slicing_dict + cp_bb

def pipeline_input_to_slicing_dict():

  # Calling the function 'luc_coordinates'
  coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

  #Calling the function 'slicing_dictionary'
  slicing_dict = slicing_dictionary(tensor)

  # Image-level: Getting the bounding box coordinates ONCE for the whole image
  corner_points_of_bounding_boxes_original_image_with_padding = {}

  for i in range(0, df.shape[0]):

    bb = i
    #function: getting_bounding_boxes
    x_min, y_min, x_max, y_max, class_object = getting_bounding_boxes(width_img, height_img, comp_size, bb)
    corner_points_of_bounding_boxes_original_image_with_padding[bb] = [x_min, y_min, x_max, y_max, class_object]

  cp_bb = corner_points_of_bounding_boxes_original_image_with_padding  # = dict
  cp_bb

  return slicing_dict, cp_bb


# Pipeline (1)

In [20]:
# Needed Variables

#for 'luc_coordinates'
num_width_comp = 8
num_height_comp = 6
comp_size = 640

#for 'slicing_dict'
tensor = np.ones((4140, 3140, 3))

#for 'getting_bounding_boxes'
width_img = 4000
height_img = 3000
#comp_size

In [ ]:
coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

slicing_dict = slicing_dictionary(tensor)
# slicing_dict, cp_bb

coordinates_dict, slicing_dict

In [22]:
# Need a function that gets me 48 DataFrames out of a 'chunk' of 48 bounding
# box coordinates. Store them in a dictionary with those DataFrames
# -> then I can work with them

ex_bbs

examp_df = pd.DataFrame(ex_bbs)
examp_df.columns = ['x_min', 'y_min', 'x_max', 'y_max']
examp_df

,x_min,y_min,x_max,y_max
0,442,434,277,102
1,354,592,344,350
2,613,119,131,467
3,267,109,271,363
4,560,103,174,70
5,510,568,595,611
6,504,466,530,395
7,194,240,477,629
8,621,79,237,468
9,225,154,406,314


In [23]:
# Create an empty DataFrame with the specified columns
columns = ['class', 'x_min', 'y_min', 'x_max', 'y_max']
data_frame = pd.DataFrame(columns=columns)
data_frame

# Running this once, so that I have a dictionary full of names for the dataframes that come out of my loops
dict_df_comps = {}

for i in range (0, 48):
    dict_df_comps[i] = examp_df

dict_df_comps

{0:    x_min  y_min  x_max  y_max
 0    442    434    277    102
 1    354    592    344    350
 2    613    119    131    467
 3    267    109    271    363
 4    560    103    174     70
 5    510    568    595    611
 6    504    466    530    395
 7    194    240    477    629
 8    621     79    237    468
 9    225    154    406    314,
 1:    x_min  y_min  x_max  y_max
 0    442    434    277    102
 1    354    592    344    350
 2    613    119    131    467
 3    267    109    271    363
 4    560    103    174     70
 5    510    568    595    611
 6    504    466    530    395
 7    194    240    477    629
 8    621     79    237    468
 9    225    154    406    314,
 2:    x_min  y_min  x_max  y_max
 0    442    434    277    102
 1    354    592    344    350
 2    613    119    131    467
 3    267    109    271    363
 4    560    103    174     70
 5    510    568    595    611
 6    504    466    530    395
 7    194    240    477    629
 8    621     79    237    4

# Re-stitching-functions

In [31]:
pred_dict = {'boxes': np.array([[[-1., -1., -1., -1.],
         [-1., -1., -1., -1.],
         [0., 2., 3., 4.],
         [5., -12., -13., -11.]],
         [[-1., -13., -1., -1.],
         [-1., -5., -2., -1.],
         [-0., -12., -1., -4.]],
         [[0., 52., -4., -1.],
         [-1., -3., -52., -1.],
         [-1., -74., -1., -1.]]]),
 'confidence': np.array([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1.]],),
 'classes': np.array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1]]),
 'num_detections': np.array([0],)}

/var/folders/kn/lz__gssj7cs776592kqhfv980000gn/T/ipykernel_51520/2246778566.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pred_dict = {'boxes': np.array([[[-1., -1., -1., -1.],


In [34]:
dict_of_dfs = {}


for i in range (0, 3): #compartments
    columns = ['x_min', 'y_min', 'x_max', 'y_max']
    data_frame = pd.DataFrame(data= pred_dict['boxes'][i], columns=columns)
    dict_of_dfs[i] = data_frame

dict_of_dfs

    # # compartment 0
    # for ind, ele in enumerate(pred_dict['boxes'][i]): #bounding boxes
    #     columns = ['x_min', 'y_min', 'x_max', 'y_max']
    #     data_frame = pd.DataFrame(columns=columns)
    #     data_frame.loc[len(data_frame)] = pred_dict['boxes'][i][ind]

    # dict_of_dfs[i] = data_frame

{0:    x_min  y_min  x_max  y_max
 0   -1.0   -1.0   -1.0   -1.0
 1   -1.0   -1.0   -1.0   -1.0
 2    0.0    2.0    3.0    4.0
 3    5.0  -12.0  -13.0  -11.0,
 1:    x_min  y_min  x_max  y_max
 0   -1.0  -13.0   -1.0   -1.0
 1   -1.0   -5.0   -2.0   -1.0
 2   -0.0  -12.0   -1.0   -4.0,
 2:    x_min  y_min  x_max  y_max
 0    0.0   52.0   -4.0   -1.0
 1   -1.0   -3.0  -52.0   -1.0
 2   -1.0  -74.0   -1.0   -1.0}

In [36]:
def re_to_absolute_coordinates_xyxy(pred_dict):
    '''
    Input:
    pred_dict = output by the prediction of the model (dictionary with one key
    'boxes')
    Calculates the absolute coordinates of a bounding box
    for a single compartment in the context of the output
    image of size width = 4000, height = 3000
    Output: absolute coordinates in form (xyxy = x_min, y_min, x_max, y_max)
    Those need to be blurred
    '''

    # Calling the function 'luc_coordinates' and saving the resulting dictionary of coordinates in a variable
    coordinates_dict = luc_coordinates()

    # Calling the function 'slicing_dict' and saving the resulting dictionary in a variable
    slicing_dict = slicing_dictionary(coordinates_dict)

    # Transforming the output by the prediction of the model to dataframes =
    # usable format to work with afterwards
    dict_of_dfs = {}

    for i in range (0, 48):
        columns = ['x_min', 'y_min', 'x_max', 'y_max']
        data_frame = pd.DataFrame(data= pred_dict['boxes'][i], columns=columns)
        dict_of_dfs[i] = data_frame

    # Defining empty dataframe with the right column-names
    new_df = pd.DataFrame(columns=['x_min', 'y_min', 'x_max', 'y_max'])

    for key, value in dict_of_dfs.items():
        # key represents the number of the compartment
        # value represents the Dataframe with bounding box coordinates

        for num in range(0, value.shape[0]):

            abs_x_min = (value.iloc[num][0] + slicing_dict[key][0]) - 70
            abs_y_min = (value.iloc[num][1] + slicing_dict[key][2]) - 70
            abs_x_max = (value.iloc[num][2] + slicing_dict[key][0]) - 70
            abs_y_max = (value.iloc[num][3] + slicing_dict[key][2]) - 70

            new_df.loc[len(new_df)] = abs_x_min, abs_y_min, abs_x_max, abs_y_max

    return new_df       # Puts out one big Dataframe with all bounding boxes
                        # to be blurred

In [37]:
new_df = re_to_absolute_coordinates_xyxy(dict_of_dfs)

In [38]:
new_df

,x_min,y_min,x_max,y_max
0,-71.0,-71.0,-71.0,-71.0
1,-71.0,-71.0,-71.0,-71.0
2,-70.0,-68.0,-67.0,-66.0
3,-65.0,-82.0,-83.0,-81.0
4,429.0,-83.0,429.0,-71.0
5,429.0,-75.0,428.0,-71.0
6,430.0,-82.0,429.0,-74.0
7,930.0,-18.0,926.0,-71.0
8,929.0,-73.0,878.0,-71.0
9,929.0,-144.0,929.0,-71.0
